# Who shall possess the one ring to rule them all (viewers)?

An utterly ridiculous study to explore and compare structured data from the top 11 gaming youtubers in Indonesia

### importing dependencies

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from googleapiclient.discovery import build
import requests
from bs4 import BeautifulSoup
import re
from datetime import timedelta
import seaborn as sns
from config import API_KEY

### The candidates

[This](https://id.noxinfluencer.com/youtube-channel-rank/top-100-id-gaming-youtuber-sorted-by-subs-weekly) data website actually provides an informative and updated data about gaming youtubers in Indonesia. However that thing isnt free and it wont even let me scrape nor copy the channel name of the top youtubers -_- . I then resort to [this](https://www.celebrities.id/read/10-youtuber-gaming-terbaik-di-indonesia-jess-no-limit-nomor-1-51XO2D) and [this](https://www.sportstars.id/read/5-youtuber-gaming-indonesia-terpopuler-no-1-legenda-mobile-legends-93s7hP?page=2) website for the list of the top 11 gaming youtuber in Indonesia. My functions are sufficiently modular, so this list could be added any time to make it more exhaustive.

In [3]:
candidate_youtubers = ["Jess No Limit", "Dyland Pros", "MiawAug", "Frost Diamond", 
                       "BUDI01 Gaming", "Kemas Pake Z mantap kali", "Rendy Rangers", 
                       "Erpan1140", "LetDa Hyper", "FrontaL Gaming", "Windah Basudara"]

### Configuring the API key and Youtube Build object

In [2]:
api_key = API_KEY
youtube = build("youtube", "v3", developerKey=api_key)

### Finding each of the channel id

I need to figure out the channel_ids so that it could be fed into my function. However, I cant seem to find any function in the youtube API to obtain the channel_ids given the name of the channel :((.

 I could just look into them one by one and probably take about 10 minutes. However, Im gonna do the pro gamer move and abide by the principle of:
__it is better to automate something for 6 six hours rather than doing it by hand for 10 minutes__ 

In [4]:
def extract_channel_id(url):
    pattern = r"channel_id=([\w-]+)"  # ChatGPT figured this out. Im no regex wizard :p
    match = re.search(pattern, url)
    return match.group(1)

def get_channel_ids(channel_names):

    youtuber_channel_id = {}

    for channel_name in channel_names:
        user = channel_name.replace(" ", "")
        channel_home_url = f"https://www.youtube.com/@{user}/featured"
        page = requests.get(channel_home_url)
        soup = BeautifulSoup(page.content, "html.parser")
        channel_id_url = soup.find("link", {"rel" : "alternate", "title":"RSS"})["href"]
        channel_id = extract_channel_id(channel_id_url) 
        youtuber_channel_id[channel_name] = channel_id

    return youtuber_channel_id

In [195]:
names_channel_id = get_channel_ids(candidate_youtubers)

df_channel_id = pd.Series(names_channel_id)
df_channel_id.to_csv("df_channel_id.csv")
df_channel_id

Jess No Limit               UCvh1at6xpV1ytYOAzxmqUsA
Dyland Pros                 UCXdmo_q4SawYMz-dmeKEHPQ
MiawAug                     UC3J4Q1grz46bdJ7NJLd4DGw
Frost Diamond               UC4hGmH5sABOA70D4fGb8qNQ
BUDI01 Gaming               UC8qp0ZmavLOGeSdLxSUZO8w
Kemas Pake Z mantap kali    UCCEs4SbttY3l73m0WsZix3g
Rendy Rangers               UCg_GNu-J5_naq-k1kUfq1Rw
Erpan1140                   UCHtehvb3p55uRCQBDCwF-ow
LetDa Hyper                 UCCsHdU66sCbEtklvSVAGPsQ
FrontaL Gaming              UCW6fcXU68QShoXPkxnPxATw
Windah Basudara             UCoIiiHof6BJ85PLuLkuxuhw
dtype: object

Now that the channel_ids are obtained, I am heading to youtube API to obtain the data

### Functions to obtain the channels' information given the channel ID

In [3]:
def get_channel_stats(channel_id):
    request = youtube.channels().list(
        part="statistics",
        id=channel_id
    )
    response = request.execute()
    stats = response["items"][0]["statistics"]
    sub_count = stats["subscriberCount"]
    view_count = stats["viewCount"]
    vid_count = stats["videoCount"]
    
    return sub_count, view_count, vid_count

def df_channel_stats(channel_ids):
    channel_name = channel_ids.index
    sub_count = []
    view_count = []
    vid_count = []
    for id_ in channel_ids.values:
        list_ = get_channel_stats(id_)
        sub_count.append(list_[0])
        view_count.append(list_[1])
        vid_count.append(list_[2])
        
    channel_stats_dict = {"channel_name":channel_name, "sub_count":sub_count,
                          "view_count":view_count, "vid_count":vid_count}
    df_channel_stats = pd.DataFrame(channel_stats_dict)
    return df_channel_stats

### The overall statistics of every channels

In [198]:
channel_ids = pd.read_csv("df_channel_id.csv", index_col = 0).squeeze()
channel_stats = df_channel_stats(channel_ids)
channel_stats.to_csv("channel_stats.csv")
channel_stats

,channel_name,sub_count,view_count,vid_count
0,Jess No Limit,27200000,3363255610,2037
1,Dyland Pros,16000000,1502425987,2686
2,MiawAug,19900000,5991542329,3657
3,Frost Diamond,27900000,6762583339,2693
4,BUDI01 Gaming,13400000,1154910766,308
5,Kemas Pake Z mantap kali,8240000,782150994,1194
6,Rendy Rangers,9440000,1782006678,2859
7,Erpan1140,8120000,1634422322,1545
8,LetDa Hyper,9550000,921263877,1224
9,FrontaL Gaming,11900000,299733499,264


In [8]:
pd.read_csv("channel_stats.csv", index_col=0)["vid_count"][0:3].sum()

8380

### Functions to obtain the videos' information given the channel ID

In [10]:
def get_channel_playlistid (youtube, channel_id):
    request = youtube.channels().list(
            part="contentDetails",
            id=channel_id)
    response = request.execute()
    return (response["items"][0]["contentDetails"]["relatedPlaylists"]["uploads"])

def get_video_id (youtube, playlist_id):

    video_ids = []
    request = youtube.playlistItems().list(
        part="Snippet",
        playlistId=playlist_id, maxResults=50)
    response = request.execute()
    for i in response["items"]:
        video_ids.append(i["snippet"]["resourceId"]["videoId"])

    next_page_token = response.get("nextPageToken")
    while next_page_token is not None:
        request = youtube.playlistItems().list(
            part="Snippet",
            playlistId=playlist_id,
            maxResults=50,
            pageToken=next_page_token)
        response = request.execute()
        for i in response["items"]:
            video_ids.append(i["snippet"]["resourceId"]["videoId"])
        next_page_token = response.get("nextPageToken")

    return video_ids

def get_df_vid_details(youtube, channel_id):

    title = []
    view_count = []
    like_count = []
    upload_date = []
    duration = []
    was_live = []
    data = get_video_id (youtube, get_channel_playlistid (youtube, channel_id))
    for i in data:
        request = youtube.videos().list(
            part="snippet, statistics, contentDetails, liveStreamingDetails",
            id=i)
        response = request.execute()
        response_item = response["items"][0]
        statistics_ = response_item["statistics"]
        snippet_ = response_item["snippet"]

        title.append(snippet_["title"])
        view_count.append(statistics_["viewCount"])
        try:
            like_count.append(statistics_["likeCount"])
        except KeyError:
            like_count.append(np.nan)
        upload_date.append(snippet_["publishedAt"])
        duration.append(response_item["contentDetails"]["duration"])
        if "liveStreamingDetails" in response_item:
            was_live.append(1)
        else:
            was_live.append(0)  
        
    result_dict = {"title":title, "view_count":view_count, "like_count":like_count, 
                   "upload_date":upload_date, "duration":duration, "was_live":was_live}
    result = pd.DataFrame(result_dict)
    return result

### Obtaining the data piece by piece

Unfortunately youtube API can only allow 10.000 list method calls, so im not able to obtain all the data at once. My rough but completely reliable :p calculation tells me that i will need 3 days to obtain all the data.

In [4]:
def get_data_by_index(channel_ids, index):
    df_master = pd.DataFrame(columns = ["channel", "title", "view_count", "like_count", "upload_date", "duration", "was_live"])
    channel_ids = channel_ids[index]
    for idx, values in channel_ids.items():
        df = get_df_vid_details(youtube, values)
        df.insert(0, "channel", idx)
        df_master = pd.concat([df_master, df], ignore_index=True)
    return df_master

### Data for index 0,1,2

In [12]:
channel_ids = pd.read_csv("df_channel_id.csv", index_col = 0).squeeze()
raw_012 = get_data_by_index(channel_ids, list(range(3)))
raw_012

,channel,title,view_count,like_count,upload_date,duration,was_live
0,Jess No Limit,Buka Tutup Botol Pakai Pisau!,31762,1842,2023-06-04T02:00:19Z,PT10S,0
1,Jess No Limit,Apakah Kalian Bisa Like dan Subscribe Pakai Hi...,101816,15918,2023-06-03T11:48:54Z,PT17S,0
2,Jess No Limit,Main Spinning Wheel Untuk Bagi HP Gratis!,74275,5021,2023-06-03T10:06:58Z,PT15S,0
3,Jess No Limit,Tebak Coklat Mana Yang Asli atau Gambar!,98712,8332,2023-06-03T04:57:45Z,PT20S,0
4,Jess No Limit,Pecahin Telur Isi Uang?,709224,45607,2023-06-02T12:24:19Z,PT24S,0
...,...,...,...,...,...,...,...
8359,MiawAug,Work Hard Play Hard - TEBAK GAMBAR - Level 2,244052,7172,2014-04-01T09:02:37Z,PT4M19S,0
8360,MiawAug,Use Your Imagination - TEBAK GAMBAR - Level 1,906709,29562,2014-04-01T02:41:14Z,PT4M26S,0
8361,MiawAug,Unlimited Challenge- SMASH HIT - Part 3,425021,9119,2014-03-28T13:54:26Z,PT12M4S,0
8362,MiawAug,Let's Dance With The Balls- SMASH HIT - Part 2,275705,10790,2014-03-28T04:15:13Z,PT12M15S,0


In [13]:
raw_012.to_csv("raw_012.csv")

### Data for index 3,4,5,6

In [7]:
channel_ids = pd.read_csv("df_channel_id.csv", index_col = 0).squeeze()
raw_3456 = get_data_by_index(channel_ids, list(range(3,7)))
raw_3456

,channel,title,view_count,like_count,upload_date,duration,was_live
0,Frost Diamond,BIKIN RUMAH SUPER AMAN VS 1000 SKIBIDI TOILET ...,7704,1092,2023-06-05T05:15:15Z,PT19M40S,0
1,Frost Diamond,Pilih 1 hadiah di gelas,232913,15112,2023-06-04T06:00:10Z,PT45S,0
2,Frost Diamond,5 TEMPAT YANG TIDAK BOLEH KAMU KUNJUNGI SENDIR...,140824,13937,2023-06-04T02:43:02Z,PT15M35S,0
3,Frost Diamond,Slime super besar 🤑,2657192,255017,2023-06-03T12:22:10Z,PT17S,0
4,Frost Diamond,Subscriber ada yang mau main ini gak? Dapet 1 ...,676191,54805,2023-06-03T03:17:44Z,PT52S,0
...,...,...,...,...,...,...,...
7064,Rendy Rangers,Sword Art Online Integral Factor Gameplay!! Tu...,13972,478,2017-12-03T11:51:04Z,PT14M7S,0
7065,Rendy Rangers,"Jurassic Survival Gameplay, New SURVIVAL GAME,...",9417,376,2017-12-03T05:51:48Z,PT9M10S,0
7066,Rendy Rangers,"Cara Rekam Screen HP kamu, gampang gak ribet !...",119740,3998,2017-11-26T07:54:40Z,PT3M17S,0
7067,Rendy Rangers,MIC MUURAH TERBAIK YOUTUBER UNBOXING SAMSON ...,195855,8657,2017-02-23T15:31:10Z,PT5M7S,0


In [8]:
raw_3456.to_csv("raw_3456.csv")

### Data for index 7,8,9,0

In [5]:
channel_ids = pd.read_csv("df_channel_id.csv", index_col = 0).squeeze()
raw_7890 = get_data_by_index(channel_ids, list(range(7,11)))
raw_7890

,channel,title,view_count,like_count,upload_date,duration,was_live
0,Erpan1140,PLAYER API GRATIS SEMAKIN DIDEPAN!!,86075,5406,2023-06-05T09:59:06Z,PT19M30S,0
1,Erpan1140,HARTA TAHTA KABIR!!,127382,7405,2023-06-04T12:56:10Z,PT23M18S,0
2,Erpan1140,MANUSIA MANUSIA ANEH…,155046,7341,2023-05-31T15:49:32Z,PT11M32S,0
3,Erpan1140,SEPEDA SIAPA TU MANNNN!!!,255042,10532,2023-05-28T10:12:42Z,PT17M40S,0
4,Erpan1140,TUTORIAL NONTON COLDPLAY GRATIS!,270135,11139,2023-05-27T07:55:47Z,PT14M4S,0
...,...,...,...,...,...,...,...
7299,Windah Basudara,BERBURU STRAWBERRY! - Celeste Lets Play! - PART 1,109524,3735,2019-01-18T06:38:53Z,PT29M40S,0
7300,Windah Basudara,Spider-Man: Far From Home | Teaser Trailer - R...,197824,10116,2019-01-17T16:19:35Z,PT6M36S,0
7301,Windah Basudara,Main MONSTER HUNTER WORLD Lagi Yuk! (Avatar: L...,278713,10756,2019-01-12T11:22:54Z,PT27M12S,0
7302,Windah Basudara,CUMA DIBAWAH 1 JUTA!? UNBOXING KURSI GAMING MU...,805594,38194,2019-01-10T09:03:19Z,PT10M57S,0


In [6]:
raw_7890.to_csv("raw_7890.csv")

### Combining the whole data

In [53]:
raw_012 = pd.read_csv("raw_012.csv", index_col=0)
raw_3456 = pd.read_csv("raw_3456.csv", index_col=0)
raw_7890 = pd.read_csv("raw_7890.csv", index_col=0)

df_raw = pd.concat([raw_012, raw_3456, raw_7890], ignore_index=True)
df_raw

,channel,title,view_count,like_count,upload_date,duration,was_live
0,Jess No Limit,Buka Tutup Botol Pakai Pisau!,31762,1842.0,2023-06-04T02:00:19Z,PT10S,0
1,Jess No Limit,Apakah Kalian Bisa Like dan Subscribe Pakai Hi...,101816,15918.0,2023-06-03T11:48:54Z,PT17S,0
2,Jess No Limit,Main Spinning Wheel Untuk Bagi HP Gratis!,74275,5021.0,2023-06-03T10:06:58Z,PT15S,0
3,Jess No Limit,Tebak Coklat Mana Yang Asli atau Gambar!,98712,8332.0,2023-06-03T04:57:45Z,PT20S,0
4,Jess No Limit,Pecahin Telur Isi Uang?,709224,45607.0,2023-06-02T12:24:19Z,PT24S,0
...,...,...,...,...,...,...,...
22732,Windah Basudara,BERBURU STRAWBERRY! - Celeste Lets Play! - PART 1,109524,3735.0,2019-01-18T06:38:53Z,PT29M40S,0
22733,Windah Basudara,Spider-Man: Far From Home | Teaser Trailer - R...,197824,10116.0,2019-01-17T16:19:35Z,PT6M36S,0
22734,Windah Basudara,Main MONSTER HUNTER WORLD Lagi Yuk! (Avatar: L...,278713,10756.0,2019-01-12T11:22:54Z,PT27M12S,0
22735,Windah Basudara,CUMA DIBAWAH 1 JUTA!? UNBOXING KURSI GAMING MU...,805594,38194.0,2019-01-10T09:03:19Z,PT10M57S,0


In [54]:
df_raw.to_csv("raw_data.csv")

### The space-and-time-consuming process of data cleaning -_-

In [65]:
MIN_VIEW_COUNT = 5  #used to ensure that the video is not private nor error

def convert_duration(youtube_duration):
    pattern = r'PT(?:(\d+)H)?(?:(\d+)M)?(?:(\d+)S)?'
    match = re.match(pattern, youtube_duration)
    
    hours = int(match.group(1)) if match.group(1) else 0
    minutes = int(match.group(2)) if match.group(2) else 0
    seconds = int(match.group(3)) if match.group(3) else 0
    
    total_minutes = hours * 60 + minutes + seconds / 60
    return round(total_minutes, 3)

def data_preprocessing(df):
    df = df.copy()
    df = df[df["view_count"]>MIN_VIEW_COUNT]
    df["view_count"] = df["view_count"].astype("int32")
    df["like_count"] = df["like_count"].fillna(-1)
    df["like_count"] = df["like_count"].astype("int32")
    df["upload_date"] = pd.to_datetime(df["upload_date"])
    df["duration_m"] = df["duration"].apply(convert_duration)
    df["date_upload"] = df["upload_date"].dt.date
    df["hour_upload"] = df["upload_date"].dt.hour
    df = df.drop(columns=["upload_date", "duration"])
    df["was_live"] = df["was_live"].astype("int8")
    return df

In [66]:
df_clean = data_preprocessing(df_raw)
df_clean

,channel,title,view_count,like_count,was_live,duration_m,date_upload,hour_upload
0,Jess No Limit,Buka Tutup Botol Pakai Pisau!,31762,1842,0,0.167,2023-06-04,2
1,Jess No Limit,Apakah Kalian Bisa Like dan Subscribe Pakai Hi...,101816,15918,0,0.283,2023-06-03,11
2,Jess No Limit,Main Spinning Wheel Untuk Bagi HP Gratis!,74275,5021,0,0.250,2023-06-03,10
3,Jess No Limit,Tebak Coklat Mana Yang Asli atau Gambar!,98712,8332,0,0.333,2023-06-03,4
4,Jess No Limit,Pecahin Telur Isi Uang?,709224,45607,0,0.400,2023-06-02,12
...,...,...,...,...,...,...,...,...
22732,Windah Basudara,BERBURU STRAWBERRY! - Celeste Lets Play! - PART 1,109524,3735,0,29.667,2019-01-18,6
22733,Windah Basudara,Spider-Man: Far From Home | Teaser Trailer - R...,197824,10116,0,6.600,2019-01-17,16
22734,Windah Basudara,Main MONSTER HUNTER WORLD Lagi Yuk! (Avatar: L...,278713,10756,0,27.200,2019-01-12,11
22735,Windah Basudara,CUMA DIBAWAH 1 JUTA!? UNBOXING KURSI GAMING MU...,805594,38194,0,10.950,2019-01-10,9


### Cleaning videos before the oldest videos among the latest videos from every channel (i know it sounds weird, but it makes sense :p)

In [27]:
oldest_latest = df_clean.groupby("channel")["date_upload"].max().min()
df_clean = df_clean[df_clean["date_upload"]<oldest_latest].reset_index(drop=True)
df_clean

,channel,title,view_count,like_count,was_live,duration_m,date_upload,hour_upload
0,Jess No Limit,ASMR Makan Makanan Warna Hijau!,3612586,182094,0,1.000,2023-05-13,10
1,Jess No Limit,Pilih Uang Atau Buku?,735920,41004,0,0.550,2023-05-12,11
2,Jess No Limit,"Subscribe Dapet Rp1,000,000!",605815,39570,0,0.583,2023-05-11,11
3,Jess No Limit,Ambil Uang Sepuasnya di Koper Dalam 3 Detik!,2573951,135660,0,0.583,2023-05-10,9
4,Jess No Limit,ASMR Susun Uang!,1910571,107261,0,0.250,2023-05-09,6
...,...,...,...,...,...,...,...,...
22424,Windah Basudara,BERBURU STRAWBERRY! - Celeste Lets Play! - PART 1,109524,3735,0,29.667,2019-01-18,6
22425,Windah Basudara,Spider-Man: Far From Home | Teaser Trailer - R...,197824,10116,0,6.600,2019-01-17,16
22426,Windah Basudara,Main MONSTER HUNTER WORLD Lagi Yuk! (Avatar: L...,278713,10756,0,27.200,2019-01-12,11
22427,Windah Basudara,CUMA DIBAWAH 1 JUTA!? UNBOXING KURSI GAMING MU...,805594,38194,0,10.950,2019-01-10,9


In [28]:
df_clean.to_csv("clean_data.csv")